# DNN Assignment
In this assignment you are working together with your teammates from the second project. You will apply your new knowledge about dense neural networks to the data from your ML project to investigate, if you can make further improvements on prediction performance. Your data is (hopefully) already cleaned and transformed (this was part of your ML project) such that you can focus fully on feeding it to your neural network. Use TensorFlow 2.x in this assignment as it makes training with real-life data much more easier with many implemented features (e.g. early-stopping, TensorBoard, regularization, etc.). 

In this notebook you will learn
- how to apply a neural network to your own data using TensorFlow 2.x
- how to tune the network and monitor learning
- how to train several networks and ensemble them into a stronger model

# Module loading
Load all the necessary packages for your assignment. We give you some modules in advance, feel free to add more, if you need them.

In [1]:
!pip install imblearn

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras import layers, regularizers, Sequential
from tensorflow.keras.layers import Dense, Dropout, Normalization, Average, Input, average

from tensorflow.keras.models import Model

print('Using TensorFlow version: %s' % tf.__version__)

Using TensorFlow version: 2.8.0


In [3]:
from sklearn.model_selection import train_test_split
from pathlib import Path
from tempfile import mkdtemp

np.set_printoptions(suppress=True, precision=3)

In [4]:
!pip install -q git+https://github.com/tensorflow/docs
    
import tensorflow_docs as tfdocs
import tensorflow_docs.modeling

#import tensorflow_docs.plots
from tensorflow_docs.plots import HistoryPlotter

In [5]:
from imblearn.under_sampling import RandomUnderSampler

In [6]:
import shutil
from pathlib import Path
from tempfile import mkdtemp
from datetime import datetime
from os.path import join

from tensorflow_docs.modeling import EpochDots
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import InverseTimeDecay
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import Accuracy, Precision, Recall, binary_accuracy

## Data loading
Load here your data from your ML project. You can use either `pandas` or `numpy` to format your data. 

In [7]:
# Load data
path = "../data/fraud_detection.csv"

if 'df' not in globals().keys():
    df = pd.read_csv(path)

COL = "Unnamed: 0"
if COL in df.columns:
    df.drop(COL, axis=1, inplace=True)


In [8]:
# drop
cols = [
    'disrict', 'creation_date', 'invoice_date', 'old_index',  # drop
    'client_id', 'months_number',                             # maybe keep?
    'client_catg', 'counter_statue', 'reading_remarque', 'counter_coefficient','counter_type'
        ]

df.drop(cols, axis=1, inplace=True, errors='ignore')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4454637 entries, 0 to 4454636
Data columns (total 14 columns):
 #   Column                Dtype
---  ------                -----
 0   region                int64
 1   target                int64
 2   tarif_type            int64
 3   counter_number        int64
 4   counter_code          int64
 5   consommation_level_1  int64
 6   consommation_level_2  int64
 7   consommation_level_3  int64
 8   consommation_level_4  int64
 9   new_index             int64
 10  year_created          int64
 11  invoice_year          int64
 12  invoice_month         int64
 13  invoice_weekday       int64
dtypes: int64(14)
memory usage: 475.8 MB


In [9]:
# isolate y
TARGET = 'target'

if TARGET in df.columns:
    sr_y = df.pop(TARGET)

## Undersample

In [10]:
X = df.values.astype(np.float64)
y = sr_y.values

X,y = RandomUnderSampler().fit_resample(X, y)

In [11]:
# split data
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    stratify=y, 
                                                    train_size=0.8)

X_train.shape, y_train.shape

((564979, 13), (564979,))

## Training
For training you need a train/val split (hopefully you did a train/test split before (and you should use the same as in your ML project to make results comparable). 

In [12]:
N_VAL = int(len(X_train) * 0.2)
N_TRAIN = len(X_train)
BATCH_SIZE = 1024
STEPS_PER_EPOCH = N_TRAIN // BATCH_SIZE
EPOCHS = 5

### Build, compile and fit your model
To become fast at retraining your (different) models it is good practice to define a function that gets fed by a **model, its name, an optimizer to use and the number of epochs** you want the model to be trained. 

If your model trains for many epochs you will receive a lot of logging from TensorFlow. To reduce the logging noise you can use a callback (provided by the `tensorflow_docs` module we installed and imported for you) named `EpochDots()` that simply prints a `.` for each epoch and a full set of metrics after a number of epochs have been trained. 

If you want to produce logs for using TensorBoard you also need to include the `callbacks.TensorBoard()`.

In [13]:


#logdir = Path(mkdtemp()) / "tensorboard_logs"
#shutil.rmtree(logdir, ignore_errors=True)

log_dir = join("tensorboard_logs", datetime.now().strftime("%Y_%m_%d_%H%M%S"))

#MONITOR = 'val_binary_crossentropy'
MONITOR = 'val_recall'

def get_callbacks(model_name):
    return [EpochDots(),
            EarlyStopping(monitor=MONITOR, patience=100),
            TensorBoard(log_dir=log_dir)]

log_dir

'tensorboard_logs/2023_03_22_202805'

You can implement your callbacks in the `model.fit()` method below.

In [14]:
lr_schedule = InverseTimeDecay(
                                0.01,
                                decay_steps=STEPS_PER_EPOCH*1000,
                                decay_rate=1)   # ... 

In [37]:
def compile_and_fit(model, model_name, optimizer=None, n_epochs=10):
    # Get optimizer
    optimizer = optimizer or Adam(learning_rate=0.001)  # or lr_schedule
    
    # model.compile
    model.compile(optimizer=optimizer,
                 loss=BinaryCrossentropy(),   # from_logits=True
                 metrics=[BinaryCrossentropy(name='binary_crossentropy'),
                          'accuracy',
                          Precision(name='precision'),
                          Recall(name='recall')
                         ])
    # summary
    model.summary()
    
    # model.fit
    history = model.fit(X_train, y_train,
                       steps_per_epoch=STEPS_PER_EPOCH,
                       epochs=n_epochs,
                       validation_split=0.2, 
                       callbacks=get_callbacks(model_name))
    
    return history

#### Build your model
You can build your model by using `tf.keras.Sequential()` that helps you to sequentially define your different layers from input to output. 

In [16]:
# standerdize the data

n = X_train.shape[1]


normalizer = Normalization(input_shape=(n,),
                            axis=-1,
                           mean=X_train.mean(axis=0),
                           variance=X_train.var(axis=0))

#normalizer = Normalization(input_shape=(n,))
#normalizer.adapt(X_train)

In [17]:

regularizer = regularizers.l2(0.001)

md = Sequential([
    normalizer,
    Dense(64, activation='relu', kernel_regularizer=regularizer),
    Dropout(0.2),    
    Dense(32, activation='elu', kernel_regularizer=regularizer),
    Dropout(0.2),
    Dense(16, activation='tanh', kernel_regularizer=regularizer),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

Metal device set to: Apple M1


2023-03-22 20:28:05.577501: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-03-22 20:28:05.577636: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [38]:
#regularizer = regularizers.l2(0.00001)

md = Sequential([
    normalizer,
    Dense(256, activation='relu', kernel_regularizer=None),   
    Dense(256, activation='relu', kernel_regularizer=None),
    Dense(256, activation='relu', kernel_regularizer=None),
    Dense(256, activation='relu', kernel_regularizer=None),
    Dense(128, activation='relu', kernel_regularizer=None),   
    Dense(64, activation='relu', kernel_regularizer=None),
    Dense(1, activation='sigmoid')
])

#### Train your model
Train your model by using your `model_compile_and_fit()` function you defined above.

In [39]:
histories = dict()

model_name = "first_model"

histories[model_name] = compile_and_fit(md, model_name, n_epochs=5)

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizatio  (None, 13)               0         
 n)                                                              
                                                                 
 dense_52 (Dense)            (None, 512)               7168      
                                                                 
 dense_53 (Dense)            (None, 512)               262656    
                                                                 
 dense_54 (Dense)            (None, 512)               262656    
                                                                 
 dense_55 (Dense)            (None, 256)               131328    
                                                                 
 dense_56 (Dense)            (None, 128)               32896     
                                                      

2023-03-22 20:35:11.089090: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


551/551 [==============================] - ETA: 0s - loss: 0.6659 - binary_crossentropy: 0.6659 - accuracy: 0.5873 - precision: 0.5876 - recall: 0.5847

2023-03-22 20:35:21.528986: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.



Epoch: 0, accuracy:0.5873,  binary_crossentropy:0.6659,  loss:0.6659,  precision:0.5876,  recall:0.5847,  val_accuracy:0.5987,  val_binary_crossentropy:0.6567,  val_loss:0.6567,  val_precision:0.5968,  val_recall:0.6117,  
551/551 [==============================] - 33s 58ms/step - loss: 0.6659 - binary_crossentropy: 0.6659 - accuracy: 0.5873 - precision: 0.5876 - recall: 0.5847 - val_loss: 0.6567 - val_binary_crossentropy: 0.6567 - val_accuracy: 0.5987 - val_precision: 0.5968 - val_recall: 0.6117
Epoch 2/5
551/551 [==============================] - 32s 59ms/step - loss: 0.6539 - binary_crossentropy: 0.6539 - accuracy: 0.6023 - precision: 0.6018 - recall: 0.6039 - val_loss: 0.6531 - val_binary_crossentropy: 0.6531 - val_accuracy: 0.5993 - val_precision: 0.5852 - val_recall: 0.6857
Epoch 3/5
325/551 [================>.............] - ETA: 4s - loss: 0.6505 - binary_crossentropy: 0.6505 - accuracy: 0.6058 - precision: 0.6047 - recall: 0.6114

KeyboardInterrupt: 

#### Evaluate your model training
TensorFlow offers now (this was more cumbersome before) a simple history plotter that you can use to plot training histories and see how the model performed on training and validation data set.

In [ ]:
plotter = HistoryPlotter(metric='binary_crossentropy', smoothing_std=10)
plotter.plot(histories)

In [ ]:
plotter = HistoryPlotter(metric='recall', smoothing_std=10)
plotter.plot(histories)

In [ ]:
plotter = HistoryPlotter(metric='accuracy', smoothing_std=10)
plotter.plot(histories)

In [ ]:

%load_ext tensorboard

In [ ]:
%tensorboard --logdir tensorboard_logs

## Model tuning
You might have no luck with your first model (most surely you did not). In this section you will apply methods you know to tune your model's performance. An obvious way of course is to change your model's architecture (removing or adding layers or layer dimensions, changing activation functions). 

However, after this you might still be able to detect some overfitting and there are some more methods you can apply to improve your neural network. Some of them are regularization, learning rate decay, early stopping, or dropout. 

If you want to add regularization you can apply directly layer-wise L2- or L1-regularization by using a layer's `kernel_regularization` argument and an appropriate regularizer from the [`tensorflow.keras.regularizers`](https://www.tensorflow.org/api_docs/python/tf/keras/regularizers) module we imported for you.  

__Optimizer schedules__<br>
Quite often your optimizer does not run efficiently through the loss function surface. Remember that theory ensures a convergence of mini-batch SGD if and only if the learing rate decreases sufficiently fast. A way to apply this to your model training is to use a learning rate scheduler (learning rate decay) that reduces the learning rate over the number of update steps. The [`tf.keras.optimizers.schedules`](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/schedules) module offers you some approaches to do that. 

Note that to apply this to your `model_compile_and_fit()` function you defined above you need to implement the learning rate schedule either in there or with a helper function that your function calls inside. 

If you want to visualize different schedulers you can define them and call them on a range of values and plot them in a line plot. 

__Early stopping__<br>
Earyl stopping is a procedure that enables you to stop your training earlier than defined by your `max_epochs` argument. It is used in practices to 
1. determine the optimal parameter vector by monitoring the validation error closely (if it rises again too much stuck with the best parameters found until then) and
2. to save expensive resources (either in terms of monetary costs or ecological costs).

To implement early stopping in TensorFlow the `tf.keras` module offers you a `callback` named [`tf.keras.callback.EarlyStopping()`](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping) that monitors for you a certain metric (it makes sense here to use a validation metric) and to stop training after a certain number of epochs with no improvement or by defining a certain `min_delta` that defines a minimum value of improvement - if below the callback stops your training. 

You can add this callback simply to the callbacks defined in your `get_callbacks()` function you defined above.

__Dropout__<br>
Dropout was one of the important developments in regularization for neural networks. It was developed by Geoffrey Hinton and his team at Toronto University. 

Dropout can be applied to each layer in your network and is implemented in `tf.keras` by an own layer named `Dropout()` awaiting a dropout rate set by you. So to introduce dropout you have to rework your model design.  

Make use of your knowledge and apply tuning techniques to improve your network. 

In [ ]:
#===========#
# YOUR CODE #   see above
#===========#

## Model ensembling
You have learned that models can be ensembled. What is possible in `scikit-learn` is also possible in TensorFlow, just a little different as it is relying on its computation graph. However, any model is callable like a `layer` by invoking it on either an `Input` or on the output of another layer. Furthermore, you can also stack outputs together.

To produce an ensemble you can define a couple of models, than use their predictions as inputs for another model and produce a final output (using `keras.Model(input, output)`). But you can also start simple and use the mean predictions over all models and then compute the `argmax()` to assign them to a class in classification (via using `layers.average([model1_preds,model2_preds,...])`). You will be surprised how well this works. 

Now implement your own ensemble to improve your work even a little more and to have something more to polish up your ML project on `GitHub` ;) 

In [ ]:
# 

n = X.shape[1]

input1 = Input(shape=(n,))
l11 = Dense(4, activation='tanh')(input1)
l12 = Dense(1, activation=None)(l12)

input2 = Input(shape=(n,))
l21 = Dense(4, activation='relu')(input2)
l22 = Dense(1, activation=None)(l21)

avg = Average()([l12, l22])
out = Dense(1)(avg)

model = Model(inputs=[input1, input2], outputs=out)

model.compile(optimizer='Adam',
             loss=BinaryCrossentropy(from_logits=True),  
             metrics=[BinaryCrossentropy(name='binary_crossentropy', from_logits=True),
                      'accuracy',
                      Precision(name='precision'),
                      Recall(name='recall')
                     ])


model.summary()

history = model.fit([X_train, X_train], 
                    [y_train, y_train],
                   steps_per_epoch=STEPS_PER_EPOCH,
                   epochs=3,
                   validation_split=0.2, 
                   callbacks=get_callbacks(model_name))

